# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests
from bs4 import BeautifulSoup
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# II. Instantiate Browser Object 

In [2]:
url = 'https://ww2.amstat.org/meetings/jsm/2008/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is
browser = webdriver.Chrome()
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [3]:
xpath_search = '/html/body/table[2]/tbody/tr[2]/td/form/div/input[6]'
xpath_session = '//tr//td[1]//a[contains(@href, "activity_details")]'
xpath_titles = '//tr//td[2]'
xpath_sponsors = '//tbody//tr//td[3]'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [4]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

### Create lists of each field needed

In [15]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles) if item.text != "Title"]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor']
session_type = [re.sub(r'(\n|\r|\t)', ' ', item.text) for item in browser.find_elements_by_xpath(xpath_type) 
                if item.text != "Type" ]
session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_session)]


In [16]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract)

True

In [17]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [18]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [19]:
df.head(1)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
0,203802,ASA Board of Directors Executive Committee Wor...,https://ww2.amstat.org/meetings/jsm/2008/onlin...,ASA Board of Directors,Other Cmte/Business,203802


In [20]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [21]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2008Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [22]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
16,CE_02C,Genetic and Microarray Data Analysis,https://ww2.amstat.org/meetings/jsm/2008/onlin...,ASA,CE Course,203571
17,CE_03C,Optimal Experimental Designs,https://ww2.amstat.org/meetings/jsm/2008/onlin...,"ASA, Section on Physical and Engineering Sciences",CE Course,203572
18,CE_04C,Regression Modeling Strategies,https://ww2.amstat.org/meetings/jsm/2008/onlin...,ASA,CE Course,203573
19,CE_05C,Hot Topics in Clinical Trials,https://ww2.amstat.org/meetings/jsm/2008/onlin...,Section on Teaching Statistics in the Health S...,CE Course,203574
20,CE_06C,Successful Data Mining in Practice,https://ww2.amstat.org/meetings/jsm/2008/onlin...,ASA,CE Course,203575
21,CE_01C,Generalized Linear Mixed Models: Theory and Ap...,https://ww2.amstat.org/meetings/jsm/2008/onlin...,ASA,CE Course,203553
39,CE_07C,Design and Analysis of Epidemiologic Studies o...,https://ww2.amstat.org/meetings/jsm/2008/onlin...,"ASA, Section on Statistics in Epidemiology",CE Course,203576
41,CE_08C,Modern Practical Bayesian Clinical Trial Design,https://ww2.amstat.org/meetings/jsm/2008/onlin...,"ASA, Section on Bayesian Statistical Science",CE Course,203577
42,CE_09C,Statistical Challenges in Proteomics,https://ww2.amstat.org/meetings/jsm/2008/onlin...,"ASA, Biometrics Section",CE Course,203578
43,CE_10C,Systematically Improving Your Professional Pra...,https://ww2.amstat.org/meetings/jsm/2008/onlin...,"Committee on Career Development, ASA",CE Course,203579


In [52]:
test = df['Session_URL'].iloc[45]
test

'https://ww2.amstat.org/meetings/jsm/2008/onlineprogram/index.cfm?fuseaction=activity_details&sessionid=203570'

In [53]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [55]:
[item.get('href') for item in soup.find_all('a') if re.search('abstract_details', str(item.get('href')))]

[]

# Run Code to extract workshops and abstract_urls

In [32]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('abstract_details', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[9].text)
            id_no = soup.find_all('tr')[5].find('strong').text.strip()

            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('abstract_details', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[5].find('strong').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [43]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

True

In [44]:
len(session_url)

2361

In [45]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

True

In [46]:
len(workshop_abstract)

262

In [47]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [48]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [49]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID
0,Optimal Ray Design for Drug Combinations,https://ww2.amstat.org/meetings/jsm/2008/onlin...,34,https://ww2.amstat.org/meetings/jsm/2008/onlin...,203518


In [50]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [51]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [42]:
df_workshops.head(3)

,Abstract_Text,Session_No,Session_URL,Session_ID
0,,"Thu, 7/31/08, 7:00 PM - 9:00 PM",https://ww2.amstat.org/meetings/jsm/2008/onlin...,203802
1,,"Fri, 8/1/08, 7:30 AM - 8:30 AM",https://ww2.amstat.org/meetings/jsm/2008/onlin...,203803
2,,"Fri, 8/1/08, 8:30 AM - 6:00 PM",https://ww2.amstat.org/meetings/jsm/2008/onlin...,203804


In [56]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [57]:
df_abstracts.to_excel(r'{}\2008Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2008Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [58]:
test = df_abstracts['Abstract_URL'].iloc[0]
test

'https://ww2.amstat.org/meetings/jsm/2008/onlineprogram/index.cfm?fuseaction=abstract_details&abstractid=302409'

In [59]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [63]:
re.sub(r'\r|\n|\t', '', soup.find_all('tr')[14].find('p').text)

'When two drugs are studied as a mixture in the drug development, one expects the combination works better than any of the drugs works alone. To investigate this, we consider the fixed-ratio ray design that the ratio of drug A to drug B is fixed on a single ray for different doses and the observation will be collected on a few rays (ratios) in the whole experiment. The model under consideration is the nonlinear logistic model and the locally D-optimal design techniques will be applied to locate the optimal ray design which answers the following questions: how many rays are needed? What ratios should one chose? How many subjects should be located on each dose? We also consider the D-optimal design under some restrictions such as the total number of rays and the number of different doses on each ray is limited. Comparison of the optimal design to a random selected ray design is also made. '

# Code to extract abstract details from abstract URLs

In [64]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('keyword', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[5].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[14].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [65]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [66]:
df_abstracts.to_excel(r'{}\2008Abstracts.xlsx'.format(dest))

In [67]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID,Abstract_ID,Abstract_Keywords,Abstract_Type,Abstract_Text
0,Optimal Ray Design for Drug Combinations,https://ww2.amstat.org/meetings/jsm/2008/onlin...,34,https://ww2.amstat.org/meetings/jsm/2008/onlin...,203518,302409,"drug combination, fixed-ratio ray design, loca...",Contributed,When two drugs are studied as a mixture in the...


In [68]:
df_details = df_workshops.append(df_abstracts)

In [69]:
df.head(0)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID


In [70]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [71]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [72]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [73]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [74]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [75]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [76]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [77]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2008_JSM_data.xlsx'.format(new_dest))